In [ ]:
%autosave 5
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

In [ ]:
file = 'Survey_Data.csv'
df = pd.read_csv(file, encoding = 'unicode_escape')
df.columns.tolist()

## Obtain binary questions

In [ ]:
# Extract columns with binary answers
binary_df = df.iloc[:,np.r_[0, 6, 7, 9:26, 43:52, 69:76, 84, 86:94, 95]]
binary_df.columns

In [ ]:
binary_df.head()

In [ ]:
# Map binary values
binary_df['Q13|Answer'] = binary_df['Q13|Answer'].map({'Yes':1.0, 'No':0.0})
binary_df['Q11|Answer 1'] = binary_df['Q11|Answer 1'].map({'Yes':1.0, 'No':0.0})
binary_df['Q3|Answer 1'] = binary_df['Q3|Answer 1'].map({'Yes':1.0, 'No':0.0})
binary_df['Q8|Answer 1'] = binary_df['Q8|Answer 1'].map({'Yes':1.0, 'No':0.0})
binary_df['Q8|Answer 2'] = binary_df['Q8|Answer 2'].map({'Yes':1.0, 'No':0.0})
binary_df['Q8|Answer 3'] = binary_df['Q8|Answer 3'].map({'Yes':1.0, 'No':0.0})
binary_df['Q8|Answer 4'] = binary_df['Q8|Answer 4'].map({'Yes':1.0, 'No':0.0})
binary_df['Q8|Answer 5'] = binary_df['Q8|Answer 5'].map({'Yes':1.0, 'No':0.0})
binary_df['Q8|Answer 6'] = binary_df['Q8|Answer 6'].map({'Yes':1.0, 'No':0.0})
binary_df['Q8|Answer 7'] = binary_df['Q8|Answer 7'].map({'Yes':1.0, 'No':0.0})                                                                    
binary_df.head()

In [ ]:
## Check output for key cols
binary_df['Q11|Answer 1'].value_counts()

### Unpivot binary

In [ ]:
# Unpivot
binary_unpivoted = binary_df.melt(id_vars=['Patient Id'], var_name='Questions', value_name='Value')
binary_unpivoted.tail()

In [ ]:
# Check output & identify unmapped values
binary_unpivoted['Value'].value_counts()
#binary_unpivoted.loc[(binary_unpivoted["Value"]== "Yes")].index.to_list()
#binary_unpivoted.loc[[76594, 76595, 76612]]

In [ ]:
# Check NAs
binary_unpivoted.isna().sum()

In [ ]:
binary_unpivoted.info()

### Aggregate binary questions

In [ ]:
binary_unpivoted.set_index(['Patient Id', 'Questions'])

#### Obtain max() aggregation

In [ ]:
binary_df_agg = binary_unpivoted.groupby(['Patient Id', 'Questions'])['Value'].max().reset_index()
binary_df_agg

## Obtain impact questions

In [ ]:
impact_df = df.iloc[:,np.r_[0, 27:43, 53:61, 61, 77:84]]
impact_df.columns

In [ ]:
impact_df.head()

In [ ]:
impact_unpivoted = impact_df.melt(id_vars=['Patient Id'], var_name='Questions', value_name='Value')
impact_unpivoted.tail()

In [ ]:
impact_unpivoted['Value'].value_counts()

### Aggregate impact columns

In [ ]:
# First set index
impact_unpivoted.set_index(['Patient Id', 'Questions'])

#### Obtain max() aggregation

In [ ]:
impact_df_agg_max = np.round(impact_unpivoted.groupby(['Patient Id', 'Questions'])['Value'].max().reset_index(), 2)
impact_df_agg_max

In [ ]:
impact_df_agg_max['Value'].value_counts()

## Obtain 'especial' column

In [ ]:
especial_df = df.iloc[:,np.r_[0, 62:69]]
especial_df.columns

In [ ]:
especial_df_unpivoted = especial_df.melt(id_vars=['Patient Id'], var_name='Questions', value_name='Value')
especial_df_unpivoted['Value'].value_counts()

#### Aggregate by min()

In [ ]:
especial_df_agg_min = especial_df_unpivoted.groupby(['Patient Id', 'Questions'])['Value'].min().reset_index()
especial_df_agg_min

In [ ]:
especial_df_agg_min['Value'].value_counts()

## Join dataframes

### Add 'Metric' and 'Category' column to each dataframe

In [ ]:
binary_df_agg.set_index(['Patient Id'])
binary_df_agg['Metric'] = "binary"
binary_df_agg['Category'] = binary_df_agg['Value'].map({1.0: 'Yes', 0.0: 'No'})         
binary_df_agg.shape[0]


In [ ]:
impact_df_agg_max.set_index(['Patient Id'])
impact_df_agg_max['Metric'] = "max"
impact_df_agg_max.shape[0] 

In [ ]:
especial_df_agg_min.set_index(['Patient Id'])
especial_df_agg_min['Metric'] = "min"
especial_df_agg_min.shape[0]
especial_df_agg_min.head()

In [ ]:
binary_df_agg.shape[0] + impact_df_agg_max.shape[0] + especial_df_agg_min.shape[0]

### Join tables

In [ ]:
df_transformed = pd.concat([binary_df_agg, impact_df_agg_max, especial_df_agg_min])
df_transformed

In [ ]:
df_transformed['Metric'].value_counts()

In [ ]:
len(df_transformed)

### Split question column

In [ ]:
df_transformed_split = pd.concat([df_transformed['Patient Id'], df_transformed['Questions'].str.split("|", 3, expand=True), df_transformed['Value'], df_transformed['Category'], df_transformed['Metric']], axis = 1) 
df_transformed_split

In [ ]:
df_transformed_split.rename({0: 'Q_number', 1:'Answer'}, axis=1, inplace=True)
df_transformed_split

In [ ]:
df_transformed_split.info()

In [ ]:
df_transformed_split['Q_number'].value_counts()

In [ ]:
df_transformed_split.reset_index()
df_transformed_split

In [ ]:
subset_Q5 = df_transformed_split.loc[(df_transformed_split["Q_number"]== "Q2")]
subset_Q5['Value'].value_counts()

### Add unique categories

#### Add categories for Q2 & Q5

In [ ]:
# Filter Q_number
subset_7cat_Q2_Q5 = df_transformed_split.loc[(df_transformed_split["Q_number"] == "Q2") | (df_transformed_split["Q_number"] == "Q5")]
subset_7cat_Q2_Q5['Q_number'].value_counts()

In [ ]:
# Set bin cut values and labels
bins = [0.99, 1.49, 2.5, 3.49, 4.5, 5.49, 6.5, 7.49]
names = ['Not at all Bothersome', 'Almost Not Bothersome', 'Somewhat Bothersome', 'Moderately Bothersome', 'Bothersome', 'Very Bothersome', 'Extremely Bothersome']

subset_7cat_Q2_Q5['Category'] = pd.cut(subset_7cat_Q2_Q5['Value'], bins, labels=names)
subset_7cat_Q2_Q5['Category'].value_counts()

#### Add categories for Q6

In [ ]:
# Filter Q_number
subset_7cat_Q6 = df_transformed_split.loc[(df_transformed_split["Q_number"] == "Q6")]
subset_7cat_Q6

In [ ]:
# Set bin cut values and labels
bins = [0.99, 1.49, 2.5, 3.49, 4.5, 5.49, 6.5, 7.49]
names = ['Not at All Anxious', 'Almost Not Anxious', 'Somewhat Anxious', 'Moderately Anxious', 'Anxious', 'Very Anxious', 'Extremely Anxious']

subset_7cat_Q6['Category'] = pd.cut(subset_7cat_Q6['Value'], bins, labels=names)
subset_7cat_Q6['Category'].value_counts()

## Create subset table and substitute in main table

In [ ]:
# Join subset table
subsets_cat = pd.concat([subset_7cat_Q2_Q5, subset_7cat_Q6])
subsets_cat

In [ ]:
#Delete rows from qol_transformed_split before joining subset with categories
df_transformed_split = df_transformed_split[df_transformed_split.Q_number != 'Q2']
df_transformed_split = df_transformed_split[df_transformed_split.Q_number != 'Q5']
df_transformed_split = df_transformed_split[df_transformed_split.Q_number != 'Q6']
df_transformed_split

In [ ]:
# Join back subset to main table
df_final = pd.concat([df_transformed_split, subsets_cat])
df_final

In [ ]:
df_final['Category'].value_counts()

In [ ]:
df_final['Value'] = df_final['Value'].apply(str)  
df_final.info() 

In [ ]:
# Print output file
df_final.to_csv('df_transformed', index=False)